In [1]:
%load_ext autoreload
%autoreload 2

import os

os.chdir('/data/core-rad/tobweber/bernoulli-mri')

import torch
from src.run import run_dataset_optim

In [2]:
cfg = {
    'dataset': 'knee',
    'dataset_root': '/data/core-rad/data',
    'batch_size': 256,
    'steps': 10000,
    'use_seg': False,
    'learning_rate': 5e-3,
    'bern_samples': 1,
    'mask_style': 'h',
    'num_workers': 16,
    'dense_target': 1 / 8,
    'dense_start': 0.10,
    'dense_end': 0.90,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'log_dir': 'logs/acdc_ensemble_8',
    'log_imgs': 10,
    'log_interval': 100,
    'seed': None,
}

In [4]:
num_members = 10
acc_facs = [4, 8, 16, 32]

for acc_fac in acc_facs:
    for i in range(1, 1 + num_members):
        cfg['dense_target'] = 1 / acc_fac
        cfg['log_dir'] = os.path.join('logs', 'knee_ensemble_1d_' + str(acc_fac) + '_m' + str(i))
            
        run_dataset_optim(cfg)

L: 6.75E-11 | D: 1.000:  10%|██▉                          | 999/10000 [03:29<31:24,  4.78it/s]


RuntimeError: quantile() input tensor is too large

# Compute Expected Mask

In [ ]:
import os
import matplotlib.pyplot as plt
from src.utils import get_top_k_mask


for acc_fac in acc_facs:
    path_stem = f'logs/knee_ensemble_1d_{acc_fac}_m'

    paths = [path_stem + str(i) + '/results.pt' for i in range(1, num_members + 1)]
    scores = [torch.load(f)['scores'][-1].cuda() for f in paths]

    scores_sum = torch.sum(torch.cat(scores), dim=(0,1))

    mask = get_top_k_mask(scores_sum, acc_fac)
    plt.imshow(mask.cpu(), cmap='gray')
    plt.show()

    new_path = f'logs/knee_ensemble_1d_{acc_fac}'
    os.makedirs(new_path, exist_ok=True)
    torch.save({
        'scores': [scores_sum.cpu()]
    }, os.path.join(new_path, 'results.pt'))